In [240]:
from bs4 import BeautifulSoup  # Importa BeautifulSoup de bs4
import pandas as pd  # Importa pandas
import requests  # Importa requests para realizar solicitudes HTTP
import re  # Importa el módulo re para expresiones regulares

In [241]:
pd.set_option('display.max_columns', None)  # Configura pandas para mostrar todas las columnas de los DataFrames

In [242]:
url = "https://moviesdatabase.p.rapidapi.com/titles"  # ENDPOINT de la API para solicitar datos
headers = {
    "X-RapidAPI-Key": "7afebd53f4msh5a5378a53b56ed5p11796bjsn19ceecfe1a26",  # Tu clave API para la autenticación
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"  # Host de la API necesario para la solicitud
}
querystring = {"genre":"Drama", "startYear":"2010", "titleType":"movie", "endYear":"2024"}  # Parámetros para la consulta de la API

In [243]:
# Define una función para hacer llamadas a la API 
def llamar_pelis(url, headers, params):
    response = requests.get(url, headers=headers, params=params)  # Realiza una solicitud GET a la API
    response.json()  # Crea variable con el contenido obtenido en formato JSON 
    return response.json()  # Devuelve los datos y la URL de la próxima página

In [244]:
json_pelis= llamar_pelis (url, headers, querystring)

In [245]:
next_url = "https://moviesdatabase.p.rapidapi.com" + json_pelis['next'] 
if 'next' in json_pelis and json_pelis['next'] is not None:# Calcula la URL de la próxima página
        print ("Has pasado a la siguiente página")
else: print ("Has llegado al final de las páginas")

Has pasado a la siguiente página


In [246]:
lista_drama = []
# Define una función para limpiar y estructurar los datos obtenidos
def limpiar_json(json_pelis):
        if 'results' in json_pelis:
            for peli in json_pelis['results']:
                nombre_peli = peli.get('titleText', {}).get('text', 'Unknown')  # Obtiene el nombre de la película
                id_peli = peli.get('id', 'Unknown')  # Obtiene el ID de la película
                lanza_peli = peli.get('releaseDate','Unknown')  # Asegura un diccionario para la fecha de lanzamiento
                año_peli = lanza_peli.get('year', 'Unknown')  # Obtiene el año de lanzamiento
                mes_peli = lanza_peli.get('month', 'Unknown')  # Obtiene el mes de lanzamiento
                lista_drama.append(("Película", nombre_peli, año_peli, mes_peli, id_peli))  # Agrega los datos a la lista
                return lista_drama  # Devuelve la lista limpia y estructurada.

In [247]:
resultados_drama = []  # Inicializa una lista para almacenar los resultados finales
json_pelis, next_page = llamar_pelis(url, headers, querystring)  # Llama a la función inicial

# Bucle para manejar la iteración por las páginas y procesar todos los resultados
while json_pelis:
    resultados_drama.extend(limpiar_json(json_pelis))  # Extiende la lista de resultados con los datos limpios
    if next_page in range (1,10):
        json_pelis, next_page = llamar_pelis(next_page, headers, querystring)  # Llama a la próxima página usando la misma querystring
    else:
        break  # Sale del bucle si no hay más páginas

ValueError: too many values to unpack (expected 2)

In [ ]:
print(f'Total de resultados: {len(resultados_drama)}')  # Imprime el total de resultados obtenidos
df_drama = pd.DataFrame(resultados_drama, columns=["Tipo", "Nombre", "Año", "Mes", "ID"])  # Crea un DataFrame con los resultados
print(df_drama.tail())  # Imprime las primeras 10 filas del DataFrame